In [71]:
!pip install git+https://github.com/fastai/fastai@2e1ccb58121dc648751e2109fc0fbf6925aa8887

!apt update && apt install -y libsm6 libxext6

  Cloning https://github.com/fastai/fastai (to revision 2e1ccb58121dc648751e2109fc0fbf6925aa8887) to /tmp/pip-req-build-mivbzcn4
  Running command git clone -q https://github.com/fastai/fastai /tmp/pip-req-build-mivbzcn4
  Running command git checkout -q 2e1ccb58121dc648751e2109fc0fbf6925aa8887


  Created wheel for fastai: filename=fastai-0.7.0-cp36-none-any.whl size=118998 sha256=84b808766a66a28023f5ad28b74f090f04c5119a6b2bf818d33b9666405e0ac2
  Stored in directory: /tmp/pip-ephem-wheel-cache-24zivix1/wheels/25/e9/9b/223e47f59c39f0bd393733db0f7efe8ec6f078d29c32198f9f
Successfully built fastai
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Err:2 http://deb.debian.org/debian stretch-updates InRelease             
  Couldn't create temporary file /tmp/apt.conf.K7FixC for passing config to apt-key
Hit:3 http://deb.debian.org/debian stretch Release                       
Err:4 http://deb.debian.org/debian stretch Release.gpg
  Couldn't create temporary file /tmp/apt.conf.oBmfnF for passing config to apt-key
Get:5 http://packages.cloud.google.com/apt cloud-sdk InRelease [6337 B]
Err:5 http://packages.cloud.google.com/apt cloud-sdk InRelease
  Couldn't create temporary file /tmp/apt.conf.fAXD8O for passing 

In [72]:
from fastai.imports import *
from fastai.structured import * 
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import os


In [73]:
os.listdir('../input/bluebook-for-bulldozers/')

['median_benchmark.csv',
 'Data Dictionary.xlsx',
 'random_forest_benchmark_test.csv',
 'TrainAndValid.csv',
 'train',
 'trainandvalid',
 'Test.csv',
 'valid',
 'Machine_Appendix.csv',
 'ValidSolution.csv',
 'Valid.csv']

In [74]:
df_raw = pd.read_csv('../input/bluebook-for-bulldozers/train/Train.csv',low_memory=False, parse_dates=['saledate'])

In [75]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

In [76]:
display_all(df_raw.tail().transpose())

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


In [77]:
df_raw.SalePrice = np.log(df_raw.SalePrice) #convert sale price to log scale because compeition evaluation metrics is RMSLE

In [78]:
add_datepart(df_raw,'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [79]:
df = pd.DataFrame({'col1': ['02/03/2017', '02/04/2017', '02/05/2017'], 'col2': ['a', 'b', 'a']})
add_datepart(df, 'col1') # inplace
df.head()

,col2,col1Year,col1Month,col1Week,col1Day,col1Dayofweek,col1Dayofyear,col1Is_month_end,col1Is_month_start,col1Is_quarter_end,col1Is_quarter_start,col1Is_year_end,col1Is_year_start,col1Elapsed
0,a,2017,2,5,3,4,34,False,False,False,False,False,False,1486080000
1,b,2017,2,5,4,5,35,False,False,False,False,False,False,1486166400
2,a,2017,2,5,5,6,36,False,False,False,False,False,False,1486252800


In [80]:
train_cats(df_raw)
df_raw.UsageBand.cat.categories
df_raw.UsageBand.cat.set_categories(['High','Medium','Low'], ordered=True, inplace=True)
df_raw.UsageBand = df_raw.UsageBand.cat.codes

AttributeError: 'CategoricalAccessor' object has no attribute 'code'

In [ ]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

In [ ]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy() 

n_valid = 12000 #same as Kaggle's test set size
n_trn = len(df) - n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)


X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
          m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train) # %time shows the time it takes to complete a task
print_score(m)

In [ ]:
test_df = pd.read_csv('../input/bluebook-for-bulldozers/Test.csv', low_memory=False, parse_dates=['saledate'])

In [ ]:
add_datepart(test_df, 'saledate')
train_cats(test_df)

test_df.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
test_df.UsageBand = test_df.UsageBand.cat.codes

test_df, y, nas = proc_df(test_df)

In [ ]:
print(set(df.columns) - set(test_df.columns))

# As a hack adding the column to the test dataset too.
test_df['auctioneerID_na'] = False

print(set(df.columns) - set(test_df.columns))

In [ ]:
predictions = m.predict(test_df)


In [ ]:
predictions

In [ ]:
submission = pd.DataFrame({'SalesID': test_df.SalesID, 'SalePrice': predictions})
submission.to_csv('submission.csv', index=False)